In [9]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import re
import time
import random
from selenium.webdriver.common.by import By

# Initiate the browser
browser  = webdriver.Chrome(ChromeDriverManager().install())

# Open the Website
browser.get('https://www.linkedin.com/login/es?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

#Login info
linkedin_username = 'vallarinopedro@gmail.com'
linkedin_password = 'vV1^7I22l#a%gYW^!Jl1'

#Input password
browser.find_element(By.XPATH,'//*[@id="username"]').send_keys(linkedin_username)
browser.find_element(By.XPATH,'//*[@id="password"]').send_keys(linkedin_password)
browser.find_element(By.XPATH,'//*[@id="organic-div"]/form/div[3]/button').click()

linkedin_job_links =   ['https://www.linkedin.com/jobs/search/?alertAction=viewjobs&currentJobId=3302691806&f_E=2%2C3%2C4&f_JT=F&f_TPR=r2592000&f_WT=2&geoId=92000000&keywords=Product%20Analyst&location=En%20todo%20el%20mundo&sortBy=R',                                            #Product Analyst
                        'https://www.linkedin.com/jobs/search/?alertAction=viewjobs&currentJobId=3307733341&f_AL=true&f_E=2%2C3%2C4&f_JT=F&f_TPR=a1665442986-&f_WT=2&keywords=Data%20Analyst&savedSearchId=1730274642&searchAlertRefId=c2107651-3613-43d7-8e29-2ea3afb4dce7',          #Data Analyst
                        'https://www.linkedin.com/jobs/search/?alertAction=viewjobs&currentJobId=3312018922&f_AL=true&f_E=2%2C3%2C4&f_JT=F&f_TPR=a1665442986-&f_WT=2&keywords=Product%20Owner&savedSearchId=1730274666&searchAlertRefId=396ebb5d-e2e9-457a-9360-bfbc4c560803',         #Product Owner
                        'https://www.linkedin.com/jobs/search/?alertAction=viewjobs&currentJobId=3311656801&f_AL=true&f_E=2%2C3%2C4&f_JT=F&f_TPR=a1665442986-&f_WT=2&keywords=Data%20Scientist&savedSearchId=1730274650&searchAlertRefId=23093265-c066-42b3-a4db-d2bdef7f737e']        #Data Scientist

# Upsizing of the windows and setting window position
browser.set_window_position(1, 1)
browser.maximize_window()

for link in linkedin_job_links:
    df = pd.DataFrame(columns=['Job Title','Job Company','Job Location','Job Description','Job Words'])
    browser.get(link)

    number_of_jobs = re.findall(r'\d+', browser.find_element(By.XPATH,'//*[@id="main"]/div/section[1]/header/div[1]/small').get_attribute('innerText'))
    number_of_jobs = ''.join(number_of_jobs)
    number_of_jobs = int(number_of_jobs)
    print('The number of jobs of this search is',number_of_jobs)

    while True:
        # For DOM load, 2 second wait time is appropiate
        time.sleep(2)

        # All job cards are loaded to then be reference points to scroll by
        jobs = browser.find_element(By.XPATH,"//ul[contains(@class,'scaffold-layout__list-container')]").find_elements_by_xpath('./*')

        # Due to lazy loading scroll down is necessary to load all the neccesary information in the HTML 
        for job in jobs:
            job.location_once_scrolled_into_view
            time.sleep(0.1)

        # This second time the jobs element has all the neccesary information inside of them
        jobs = browser.find_element(By.XPATH,"//ul[contains(@class,'scaffold-layout__list-container')]").find_elements_by_xpath('./*')

        for job in jobs:
            insert_row = {
                        'Job Title':       job.get_attribute('innerText').split('\n')[0],
                        'Job Company':     job.get_attribute('innerText').split('\n')[1],
                        'Job Location':    job.get_attribute('innerText').split('\n')[2],
                        'Job Description': browser.find_element(By.ID,'job-details').get_attribute('innerText'),
                        'Job Words':       browser.find_element(By.ID,'job-details').get_attribute('innerText').split() 
                        }

            df = pd.concat([df, pd.DataFrame([insert_row])])
            
        pagination_elements = browser.find_elements(By.XPATH,"//li[contains(@class,'artdeco-pagination__indicator artdeco-pagination__indicator--number')]")

        # Actual page analysis
        for page_number in pagination_elements:
            if page_number.find_element(By.CSS_SELECTOR,'button').get_attribute('aria-current') == 'true':
                current_page = page_number
                print('The current page is', page_number.get_attribute('data-test-pagination-page-btn'))

            try:
                if int(re.findall(r'\d+',page_number.find_element(By.CSS_SELECTOR,'button').get_attribute('aria-label'))[0]) == int(re.findall(r'\d+',current_page.find_element(By.CSS_SELECTOR,'button').get_attribute('aria-label'))[0]) + 1:
                    page_number.click()
                    break
            except:
                pass

The number of jobs of this search is 16429
The current page is 1
The current page is 2
The current page is 3
The current page is 4
The current page is 5
The current page is 6
The current page is 7
The current page is 8
The current page is 9
The current page is 10
The current page is 11


KeyboardInterrupt: 

In [8]:
int(re.findall(r'\d+',page_number.find_element(By.CSS_SELECTOR,'button').get_attribute('aria-label'))[0])

40